In [1]:
import param
class B(param.Parameterized):
    def identity(x): return x
    
def print_time_of_day():
    print(datetime.date.today())

class A(param.Parameterized):
    transformer = param.Callable(B.identity)
    a = param.Action(print_time_of_day)
    
    def __call__(self, x):
        return self.transformer(x)
 
a = A()
a(5)

5